### ArcFace, CosFace and SphereFace Classes

In [4]:
from keras import backend as K
from keras.layers import Layer
from keras import regularizers

import tensorflow as tf


class ArcFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class SphereFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=1.35, regularizer=None, **kwargs):
        super(SphereFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(SphereFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(self.m * theta)
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


class CosFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.35, regularizer=None, **kwargs):
        super(CosFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(CosFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        target_logits = logits - self.m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

### Plotting the Clusters after dimensionality reduction

In [6]:
def plot_embedded(dataframe, points_plotted=50):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1, 1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets = [0, 1, 2, 3, 4,5, 6, 7, 8, 9]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    for target, color in zip(targets,colors):
        indicesToKeep = dataframe['label'] == target
        #print(indicesToKeep)
        ax.scatter(list(dataframe.loc[indicesToKeep, 'pca1'])[:points_plotted]
                   ,list(dataframe.loc[indicesToKeep, 'pca2'])[:points_plotted]
                   , c = color
                   , s = 50)
    ax.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    ax.grid()

In [4]:
def plot_embedded_smaller(dataframe, points_plotted=50):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1, 1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets = [3, 2, 1, 0, 9, 6, 8, 5, 7, 4]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    for target, color in zip(targets,colors):
        indicesToKeep = dataframe['label'] == target
        #print(indicesToKeep)
        ax.scatter(list(dataframe.loc[indicesToKeep, 'pca1'])[:points_plotted]
                   ,list(dataframe.loc[indicesToKeep, 'pca2'])[:points_plotted]
                   , c = color
                   , s = 50)
    ax.legend(['dog_bark', 'children_playing', 'car_horn', 'air_conditioner',
       'street_music', 'gun_shot', 'siren', 'engine_idling', 'jackhammer',
       'drilling'])
    ax.grid()

### Calculating Cosine Similarity 

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [8]:
def get_numberOfClassesCount(y_true):
    record={}
    for i in range(len(y_true)):
        if str(y_true[i]) not in record.keys():
            record[str(y_true[i])]= 1
        else:
            record[str(y_true[i])] +=1
    return dict(sorted(record.items()))

In [9]:
def calculate_embeddings_mean(embedding_vector, y_test):
    size = embedding_vector.shape[0]
    y_true=np.argmax(y_test, axis=1)
    mean_dict = {}
    for label in list(np.unique(y_true)):
        #label = y_true[i]
        if str(label) not in mean_dict.keys():
            mean_dict[str(label)] = 0
        indexOfSameLabelFound= np.squeeze(np.array(np.where(y_true==label)))
        labels_mean=[embedding_vector[index] for index in indexOfSameLabelFound]
        labels_mean = np.array(labels_mean)
        total_sum=np.cumsum(labels_mean, axis=0)[len(labels_mean)-1]
        mean = total_sum/len(labels_mean)
        mean_dict[str(label)]=mean
    
    return mean_dict
            

In [10]:
def get_OpensetAccuracy(inter_model, x_test, y_test):
    embedded_vector = inter_model.predict(x_test, verbose=1)
    embedded_vector /= np.linalg.norm(embedded_vector, axis=1, keepdims=True)
    y_true = np.argmax(y_test, axis=1)
    class_count = get_numberOfClassesCount(y_true)
    mean_embeddings = calculate_embeddings_mean(embedded_vector, y_test)
    best_simis = []
    for label, count in class_count.items():
        all_similarities=[]
        for i in range(y_test.shape[0]):
            new_vector = embedded_vector[i]
            new_label = y_true[i]
            sim = get_cosine_similarity(mean_embeddings[label], new_vector)
            all_similarities.append([sim, new_label, label])
        sorted_simi = sorted(all_similarities, key= lambda x: x[0] , reverse=True)
        best_simis.append(sorted_simi[:count])
    
    #sim_df = pd.DataFrame(data=best_simis, columns=("cosine_similarity", "predicted_label", "ori_label"))
    #print(sim_df.head())
    match_count = 0
    for label in best_simis:
        count = 0
        label = np.array(label)
        for predict in label:
            if predict[1]==predict[2]:
                count+=1
        match_count += count
        
    return match_count/y_test.shape[0], best_simis